In [7]:
from src.data_preprocessor import DataProcessor
from data_configs.configs import *
from models.decision_tree import DecisionTree, DecisionTreeNode
from models.null_model import NullModelClassification, NullModelRegression
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
import numpy as np

config = machine_config
decision_tree = DecisionTree(config=config)
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)
null_model = NullModelClassification(config=config)

### Data Load and Preprocessing ###

In [8]:
raw_data = data_processor.load_data()

data_1 = data_processor.impute_missing_values(raw_data)

data_2 = data_1.drop(columns=['vendor_name', 'model_name','ERP'])


In [9]:
data_2.describe()

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP
count,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000
mean,203.822967,2867.980861,11796.153110,25.205742,4.698565,18.267943,105.622010
std,260.262926,3878.742758,11726.564377,40.628722,6.816274,25.997318,160.830733
min,17.000000,64.000000,64.000000,0.000000,0.000000,0.000000,6.000000
25%,50.000000,768.000000,4000.000000,0.000000,1.000000,5.000000,27.000000
50%,110.000000,2000.000000,8000.000000,8.000000,2.000000,8.000000,50.000000
75%,225.000000,4000.000000,16000.000000,32.000000,6.000000,24.000000,113.000000
max,1500.000000,32000.000000,64000.000000,256.000000,52.000000,176.000000,1150.000000


In [10]:
data_train, data_val = cross_validator.random_partition(data_2, random_state=42)

## Decision Tree ##

In [47]:
mse_scores = []
mae_scores = []
r2_scores = []
pearson_scores = []

pruned_mse_scores = []
pruned_mae_scores = []
pruned_r2_scores = []
pruned_pearson_scores = []

null_model_mse_scores = []
null_model_mae_scores = []
null_model_r2_scores = []
null_model_pearson_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

    train_data = train_set.drop(columns=config['target_column'])
    train_target = train_set[config['target_column']]
    test_features = test_set.drop(columns=config['target_column'])
    test_true_vals = test_set[config['target_column']]

    decision_tree = DecisionTree(config)
    decision_tree.root = decision_tree.build_regression_tree(train_data, train_target)
    predictions = decision_tree.predict(test_features)

    mse_score = Evaluation().mean_squared_error(test_true_vals, predictions)
    mse_scores.append(mse_score)

    decision_tree.prune(decision_tree.root, data_val)
    pruned_predictions = decision_tree.predict(test_set)
    
    pruned_mse_score = Evaluation().mean_squared_error(test_true_vals, pruned_predictions)
    pruned_mse_scores.append(pruned_mse_score)


average_mse_score = sum(mse_scores) / len(mse_scores)
average_pruned_mse_score = sum(pruned_mse_scores) / len(pruned_mse_scores)

print(f"Average Decision Tree MSE score: {average_mse_score}")
print(f"Average Pruned Decision Tree MSE score: {average_pruned_mse_score}")



Average Decision Tree MSE score: 7077.102086775674
Average Pruned Decision Tree MSE score: 6847.4062757538095


In [12]:
# Initialize lists to store scores for decision tree, pruned decision tree, and null model
dt_scores = {'mse': [], 'mae': [], 'r2': [], 'pearson_correlation': []}
pruned_dt_scores = {'mse': [], 'mae': [], 'r2': [], 'pearson_correlation': []}
null_model_scores = {'mse': [], 'mae': []}  

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
    train_data = train_set.drop(columns=config['target_column'])
    train_target = train_set[config['target_column']]
    test_features = test_set.drop(columns=config['target_column'])
    test_true_vals = test_set[config['target_column']]

    # Decision Tree Model
    decision_tree = DecisionTree(config)
    decision_tree.root = decision_tree.build_regression_tree(train_data, train_target)
    predictions = decision_tree.predict(test_features)

    # Calculate and store decision tree scores
    scores = Evaluation.calculate_regression_scores(test_true_vals, predictions)
    for key in dt_scores:
        dt_scores[key].append(scores[key])

    # Pruning the Decision Tree
    decision_tree.prune(decision_tree.root, data_val)  # Ensure data_val is correctly defined as your validation set
    pruned_predictions = decision_tree.predict(test_features)

    # Calculate and store pruned decision tree scores
    pruned_scores = Evaluation.calculate_regression_scores(test_true_vals, pruned_predictions)
    for key in pruned_dt_scores:
        pruned_dt_scores[key].append(pruned_scores[key])

    # Null Model
    null_model = NullModelRegression(config=config)
    null_model_prediction = null_model.naive_regression(test_set)

    # Calculate and store null model scores (excluding R2 and Pearson)
    null_model_mse = Evaluation.mean_squared_error(test_true_vals, null_model_prediction)
    null_model_mae = Evaluation.mean_absolute_error(test_true_vals, null_model_prediction)
    null_model_scores['mse'].append(null_model_mse)
    null_model_scores['mae'].append(null_model_mae)

# Calculate average scores for each model
average_dt_scores = {metric: np.mean(values) for metric, values in dt_scores.items()}
average_pruned_dt_scores = {metric: np.mean(values) for metric, values in pruned_dt_scores.items()}
average_null_model_scores = {metric: np.mean(values) for metric, values in null_model_scores.items()}

# Print average scores
print("Average Decision Tree Scores:")
for metric, avg_score in average_dt_scores.items():
    print(f"{metric}: {avg_score}")

print("\nAverage Pruned Decision Tree Scores:")
for metric, avg_score in average_pruned_dt_scores.items():
    print(f"{metric}: {avg_score}")

print("\nAverage Null Model Scores:")
for metric, avg_score in average_null_model_scores.items():
    print(f"{metric}: {avg_score}")


c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning

Average dt scores:
mse: 7077.102086775674
mae: 38.49862545419775
r2: 0.6411596863377425
pearson_correlation: 0.8205723803138649

Average pruned_dt scores:
mse: 6847.4062757538095
mae: 38.24156865557468
r2: 0.6598154991168503
pearson_correlation: 0.8352280430637782

Average null_model scores:
mse: 19073.865381296106
mae: 86.17113664476774



c:\Users\Srikanta\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [30]:
data = data_train.drop(columns=config['target_column'])
target = data_train[config['target_column']]

best_feature, best_mse, best_threshold = decision_tree.select_feature_mse(data, target)

decision_tree = DecisionTree(config)
decision_tree.root = decision_tree.build_regression_tree(data, target)


In [31]:
# data = data_train.drop(columns=config['target_column'])
# target = data_train[config['target_column']]

# decision_tree = DecisionTree(config)
# decision_tree.root = decision_tree.build_classification_tree(data, target)

In [32]:
test_features = data_val.drop(columns=config['target_column'])

null_model = NullModelRegression(config=config)
null_predictions = null_model.naive_regression(data_val)
true = data_val[config['target_column']]
null_score = Evaluation().mean_squared_error(true,null_predictions)

print(null_score)

predictions = decision_tree.predict(test_features)
score = Evaluation().mean_squared_error(true,predictions)
print(score)


11787.762120710055
12838.225388977686


In [33]:
decision_tree.prune(decision_tree.root, data_val)

In [34]:
test_features = data_val.drop(columns=config['target_column'])

null_model = NullModelRegression(config=config)
null_predictions = null_model.naive_regression(data_val)
true = data_val[config['target_column']]
null_score = Evaluation().mean_squared_error(true,null_predictions)

print(null_score)

predictions = decision_tree.predict(test_features)
score = Evaluation().mean_squared_error(true,predictions)
print(score)


11787.762120710055
11632.16953166397


In [35]:
# print(predictions.unique())

In [36]:
# test_features = data_val.drop(columns=config['target_column'])
# predictions = decision_tree.predict(test_features)
# true = data_val[config['target_column']]

# null_predictions = null_model.naive_classifier(data_val)

# null_eval = Evaluation().zero_one_loss(true,null_predictions)

# tree_eval = Evaluation().zero_one_loss(true,predictions)


# print(null_eval)
# print(tree_eval)